# SPBM tessellation picture

## Outline
1. Sample the positions and growth rates / random radii of the seeds.
2. Assign each pixel in an image to its appropriate cell, based on which seed reaches it first.
3. Draw the picture.

In [ ]:
import numpy as np
from tqdm import trange, tqdm
from PIL import Image, ImageDraw
from draw_jm import get_ball_pixels#, assign_cells_random_radii
from datetime import datetime

### Sampling arrival times and pruning

In [ ]:
def sample_disc_points(sample_size):
    # Chooses points uniformly in disc centered at (1/2,1/2) with radius 0.45 .
    theta = 2*np.pi*np.random.random(size=(sample_size,1))
    radius_sqrt = np.sqrt(np.random.random(size=(sample_size,1)))
    x = 0.45*radius_sqrt * np.cos(theta) + 0.5
    y = 0.45*radius_sqrt * np.sin(theta) + 0.5
    return np.concatenate((x,y),axis=1)

def assign_cells_random_radii(seeds, rates, img_size, T=1.0):
    min_cov_times = np.full((img_size,img_size),np.inf) # running minimum coverage times
    assignments = np.full((img_size,img_size),-1,dtype=int)
    for i in range(len(rates)):
        xi = seeds[i]
        gi = rates[i]
        gi2 = gi*gi
        indices, d2s = get_ball_pixels(xi, T*gi, img_size)
        for k, ij_pair in enumerate(indices):
            cov_time2 = d2s[k] / gi2
            if cov_time2 < min_cov_times[ij_pair]:
                assignments[ij_pair] = i
                min_cov_times[ij_pair] = cov_time2
    if -1 in assignments: # i.e. if there were uncovered points.
        print("Uncovered points found - trying again with a bigger radius.")
        return assign_cells_random_radii(seeds,rates,img_size,2*T) # This is slow compared to Moulinec's method of assigning the remaining unassigned pixels individually.
    return assignments, min_cov_times

In [ ]:
arrival_rate = 15 # The rate at which points arrive in the disc, equivalent to rho * |A| in the paper.
n = np.random.poisson(arrival_rate)
max_time = 2*( np.log(n) / (np.pi*n) )**(1/2)

seeds = sample_disc_points(n)
rates = np.random.exponential(size=n)

### Assigning pixels to their cells

We use a modified version of the algorithm from ["A simple and fast algorithm for computing discrete Voronoi, Johnson-Mehl or Laguerre diagrams of points"](https://www.sciencedirect.com/science/article/pii/S0965997822000618). See our file `jm-picture-with-boundaries.ipynb` for a simple summary of how the algorithm works.

In [ ]:
img_size = 2000

# Compute the assignment and coverage time of each pixel,
# and the location of the last point covered.

I_full, cov_times = assign_cells_random_radii(seeds, rates, img_size, T=max_time)
# print(I)
A,_ = get_ball_pixels(np.array([0.5,0.5]), 0.45, img_size)
I = np.full((img_size,img_size),-1,dtype=int)
max_cov_time = 0
max_time_location = (0,0)
for ij in A:
    I[ij] = I_full[ij]
    if cov_times[ij] > max_cov_time:
        max_cov_time = cov_times[ij]
        max_time_location = ij

### Finding the boundaries

We'll go through each pixel in the image and check if it is near more than one cell.
If it is then we declare it part of a boundary and colour it black.
This is a very inefficient way to do it.
Thinner boundaries are computed faster.

In [ ]:
boundary_thickness = 0.0015
# Go through I and if any point has more than one cell nearby to it, colour it black.
in_boundary = np.full((img_size,img_size),0,dtype=int)
spacing = np.linspace(0,1,num=img_size,endpoint=True)
for i, x in enumerate(tqdm(spacing)):
    for j, y in enumerate(spacing):
        v = np.array([x,y])
        local_ball,_ = get_ball_pixels(v,boundary_thickness,img_size)
        cells = { I[ij] for ij in local_ball }
        if len(cells) >= 2:
            in_boundary[i,j] = 1

In [ ]:
def seed_colour(i):
    # Returns the colour of the ith seed.
    # It just returns red, but we've left it here
    # to match the code for the JM diagrams.
    return (255, 0, 0)

def seed_size(i):
    # Returns the size of the ith seed.
    # Faster-growing seeds should be larger.
    speed = rates[i]
    max_speed = np.max(rates)
    p = speed/max_speed
    return (0.02*p + 0.005*(1-p))*img_size

In [ ]:
cell_colour = (255,255,255)
boundary_colour = (0,0,0)
box_size = 0.03*img_size

data = np.full((img_size, img_size, 3),255, dtype=np.uint8)
image = Image.fromarray(data)
draw = ImageDraw.Draw(image)
for i,v in enumerate(seeds):
    x,y = (img_size-1)*v
    draw.ellipse([y-0.5*seed_size(i),x-0.5*seed_size(i),
                  y+0.5*seed_size(i),x+0.5*seed_size(i)],
                 fill=seed_colour(i))
N = I.shape[0]
for i in range(N):
    for j in range(N):
        if (i - 0.5*(img_size-1))**2 + (j - 0.5*(img_size-1))**2 > (0.45*(img_size))**2:
            draw.point([j,i],fill=cell_colour)
        if in_boundary[i,j]:
            draw.point([j,i],fill=boundary_colour)

draw.rectangle([max_time_location[1]-0.5*box_size,max_time_location[0]-0.5*box_size,
                max_time_location[1]+0.5*box_size,max_time_location[0]+0.5*box_size],outline="blue",width=10)

# image.show() # opens in system image viewer
display(image)

In [ ]:
# image.show()
now = datetime.now().isoformat()
image.save(f"pictures/spbm-tessellation-{now}.png")

## All that, but in a loop

In [ ]:
while True:
    arrival_rate = 15 # The rate at which points arrive in the disc, equivalent to rho * |A| in the paper.
    n = np.random.poisson(arrival_rate)
    max_time = 2*( np.log(n) / (np.pi*n) )**(1/2)
    seeds = sample_disc_points(n)
    rates = np.random.exponential(1.0/0.45,size=n)
    img_size = 2000

    I_full, cov_times = assign_cells_random_radii(seeds, rates, img_size, T=max_time)
    A,_ = get_ball_pixels(np.array([0.5,0.5]), 0.45, img_size)
    I = np.full((img_size,img_size),-1,dtype=int)
    max_cov_time = 0
    max_time_location = (0,0)
    for ij in A:
        I[ij] = I_full[ij]
        if cov_times[ij] > max_cov_time:
            max_cov_time = cov_times[ij]
            max_time_location = ij

    boundary_thickness = 0.0015
    # Go through I and if any point has more than one cell nearby to it, colour it black.
    in_boundary = np.full((img_size,img_size),0,dtype=int)
    spacing = np.linspace(0,1,num=img_size,endpoint=True)
    for i, x in enumerate(tqdm(spacing)):
        for j, y in enumerate(spacing):
            v = np.array([x,y])
            local_ball,_ = get_ball_pixels(v,boundary_thickness,img_size)
            cells = { I[ij] for ij in local_ball }
            if len(cells) >= 2:
                in_boundary[i,j] = 1

    def seed_colour(i):
        # Returns the colour of the ith seed.
        return (255, 0, 0)

    def seed_size(i):
        # Returns the size of the ith seed.
        # Faster-growing seeds should be larger.
        speed = rates[i]
        max_speed = np.max(rates)
        p = speed/max_speed
        return (0.02*p + 0.005*(1-p))*img_size

    cell_colour = (255,255,255)
    boundary_colour = (0,0,0)
    #seed_size = 0.01*img_size
    box_size = 0.03*img_size

    data = np.full((img_size, img_size, 3),255, dtype=np.uint8)
    image = Image.fromarray(data)
    draw = ImageDraw.Draw(image)
    for i,v in enumerate(seeds):
        x,y = (img_size-1)*v
        s = seed_size(i)
        draw.ellipse([y-0.5*s,x-0.5*s,
                      y+0.5*s,x+0.5*s],
                     fill=seed_colour(i))
    N = I.shape[0]
    for i in range(N):
        for j in range(N):
            if (i - 0.5*(img_size-1))**2 + (j - 0.5*(img_size-1))**2 > (0.45*(img_size))**2:
                draw.point([j,i],fill=cell_colour)
            if in_boundary[i,j]:
                draw.point([j,i],fill=boundary_colour)

    draw.rectangle([max_time_location[1]-0.5*box_size,max_time_location[0]-0.5*box_size,
                    max_time_location[1]+0.5*box_size,max_time_location[0]+0.5*box_size],outline="blue",width=10)

    now = datetime.now().isoformat()
    image.save(f"pictures/spbm-tessellation-{now}.png")